Recreate testing environment if needed

```yml
name: cmaps
channels:
  - conda-forge
  - defaults
dependencies:
  - python=3.13
  - cmaps=2.0.1
  - plotly
  - nbformat
```

Plotly requires its colormaps in different form than matplotlib: https://plotly.com/python/colorscales/

In [7]:
import numpy as np
import cmaps
import plotly.graph_objects as go

fig = go.Figure(data =
     go.Contour(
        z=[[10, 10.625, 12.5, 15.625, 20],
           [5.625, 6.25, 8.125, 11.25, 15.625],
           [2.5, 3.125, 5., 8.125, 12.5],
           [0.625, 1.25, 3.125, 6.25, 10.625],
           [0, 0.625, 2.5, 5.625, 10]],
        colorscale=cmaps.BlRe.plotly,
    ), layout=dict(width=500, height=500))
fig.show()

In [8]:
''' 
Based on https://plotly.com/python/builtin-colorscales/

Adapted from https://github.com/plotly/plotly.py/blob/main/_plotly_utils/colors/_swatches.py 
'''

def swatches_continuous(cmap_names):
    import plotly.graph_objs as go

    n = 255

    return go.Figure(
        data=[
            go.Bar(
                orientation="h",
                y=[name] * n,
                x=[1] * n,
                customdata=[(x + 1) / n for x in range(n)],
                marker=dict(color=list(range(n)), colorscale=eval(f'cmaps.{name}.plotly'), line_width=0),
                hovertemplate="%{customdata}",
                name=name,
            )
            for name in reversed(cmap_names)
        ],
        layout=dict(
            title="cmaps",
            barmode="stack",
            barnorm="fraction",
            bargap=0.3,
            showlegend=False,
            xaxis=dict(range=[-0.02, 1.02], showticklabels=False, showgrid=False),
            height=max(600, 40 * len(cmap_names)),
            width=1000,
            margin=dict(b=10),
        ),
    )


def swatches_cyclical(cmap_names):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    cols = 6
    rows = len(cmap_names) // cols + 1

    fig = make_subplots(
        rows=rows,
        cols=cols,
        subplot_titles=cmap_names,
        specs=[[{"type": "polar"}] * cols] * rows,
    )

    for i, name in enumerate(cmap_names):
        fig.add_trace(
            go.Barpolar(
                r=[1] * int(360 / 5),
                theta=list(range(0, 360, 5)),
                marker_color=list(range(0, 360, 5)),
                marker_cmin=0,
                marker_cmax=360,
                marker_colorscale=eval(f'cmaps.{name}.plotly'),
                name=name,
            ),
            row=int(i / cols) + 1,
            col=i % cols + 1,
        )
    fig.update_traces(width=5.2, marker_line_width=0, base=0.5, showlegend=False)
    fig.update_polars(angularaxis_visible=False, radialaxis_visible=False)
    fig.update_layout(title="cmaps", height=250*rows)
    return fig

In [9]:
cmaps_list = list()
for x in dir(cmaps):
    if 'plotly' in dir(eval(f'cmaps.{x}')) and not x.endswith('_r'):
        cmaps_list.append(x)
        
print(len(cmaps_list), 'colormaps are in the package')

278 colormaps are in the package


In [ ]:
from random import choices

# Showing some random set of colormaps

fig = swatches_continuous(choices(cmaps_list, k=10))
fig.show()

fig = swatches_cyclical(choices(cmaps_list, k=6*3))
fig.show()

Beware! Execution of two cells below requires a lot of RAM!

In [ ]:
fig = swatches_continuous(cmaps_list)
fig.show()

In [ ]:
fig = swatches_cyclical(cmaps_list)
fig.show()